In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 6.7 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [5]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://unriffled-haematoid-rochel.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://unriffled-haematoid-rochel.ngrok-free.dev


True

In [6]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [7]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [9]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於臺灣新竹縣新豐鄉的私立科技大學。學校秉持「堅毅、求新、創造」的校訓，以培養具備專業技能、團隊精神、人文素養與宏觀服務的科技人才為願景目標。

**歷史沿革：**
明新科技大學創立於1966年（民國55年）3月1日，最初以「明新工業專科學校」立案，設有機械、土木、工業管理三科，並招收三百名新生。
*   1997年（民國86年）7月，經教育部核准改制為「明新技術學院」並附設專科部。
*   2002年（民國91年）8月，奉教育部核准優先升格為「明新科技大學」。
*   2019年（民國108年）3月，奉教育部核准更名為「明新學校財團法人明新科技大學」。

**學院與學系：**
明新科技大學現設有六大學院，包括：
*   半導體學院：整合電機工程系、電子工程系、化學工程與材料科技系、光電工程系等。
*   工程學院。
*   管理學院。
*   民生學院 (部分資料顯示為服務事業學院)。
*   人文與設計學院。
*   共同教育學院。

學校涵蓋20個學系、2個學位學程（包含1個博士學位學程）以及11個碩士班。在2021年，明新科大成立了半導體學院，並於2022年獲教育部核准通過「半導體科技博士學位學程」，這是該校成立56年來的第一個博士班。

**地理位置與特色：**
明新科技大學位於新竹縣新豐鄉，佔地逾三十公頃，交通便利，毗鄰省道縱貫線與中山高速公路。學校致力於培養「跨域整合、務實創新與全人學習」的專業人才，並以成為「一流產業大學」為發展願景。

學校也積極與產業接軌，發展「永續智慧商務」的教學與實習場域，導入生成式AI與AI專案應用，發展智慧零售、智慧金融、智慧製造與智慧商業等實驗室。明新科大在2022年《遠見》雜誌的「企業最愛公私立技職科大調查」中，獲得兩項企業最愛，起薪排名私校第一，優於國立科大。學校也積極推動國際交流，與多所國外大學簽訂合作備忘錄，提升學生的國際移動力與全球學習視野.


In [10]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長為**呂明峯**教授。

呂明峯教授於2025年2月1日正式上任，並於2025年1月16日舉行了校長佈達暨交接典禮，由董事長郝碧蓮監交，從卸任的林啓瑞校長手中接下印信，完成校務傳承。

呂明峯校長在業界擁有豐富的半導體教育與產業實務經驗，曾任電子工程系系主任、研發長、工學院院長、半導體學院院長以及產學長。他在任內曾開辦全國首見的「2+2N封裝測試產業精英專班」，並爭取預算打造全台首座半導體封裝測試類產線，推動成立半導體學院及半導體科技博士學位學程，使明新科大在半導體教育領域成為重要的一環。

他將以「四大核心模組」作為校務治理策略，帶領明新科大開創技職教育新局，並以打造國際魅力產業大學為目標。


In [ ]:
from flask import Flask, request, abort
import logging
import os
import time
from google.genai import types

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FileMessageContent
)

app = Flask(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
app.logger.setLevel(logging.INFO)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

# 儲存檔案的目錄
UPLOAD_DIR = "/content/uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# 儲存每個使用者的對話 session 和上傳的檔案
user_sessions = {}  # {user_id: {"chat": chat_object, "uploaded_file": gemini_file}}

def get_user_session(user_id):
    """取得或建立使用者的對話 session"""
    if user_id not in user_sessions:
        # 建立新的對話 session
        new_chat = client.chats.create(
            model="gemini-2.5-flash",
            config=GenerateContentConfig(
                system_instruction="你是一個中文的AI助手，請用繁體中文回答。如果使用者有提供參考文件，請根據文件內容回答問題。",
                tools=[google_search_tool],
                response_modalities=["TEXT"],
            )
        )
        user_sessions[user_id] = {
            "chat": new_chat,
            "uploaded_file": None
        }
    return user_sessions[user_id]

def download_line_file(message_id, file_name):
    """從 LINE 下載使用者上傳的檔案"""
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        file_content = line_bot_blob_api.get_message_content(message_id)

        file_path = os.path.join(UPLOAD_DIR, file_name)

        with open(file_path, 'wb') as f:
            f.write(file_content)

        return file_path

def upload_file_to_gemini(file_path):
    """上傳檔案到 Gemini Files API"""
    uploaded_file = client.files.upload(
        file=file_path,
        config={'display_name': os.path.basename(file_path)}
    )

    # 等待檔案處理完成
    while uploaded_file.state.name == "PROCESSING":
        print("檔案處理中...")
        time.sleep(1)
        uploaded_file = client.files.get(name=uploaded_file.name)

    if uploaded_file.state.name == "FAILED":
        raise Exception("檔案上傳處理失敗")

    return uploaded_file

def query_with_rag(user_id, question):
    """使用 RAG 模式回答問題"""
    session = get_user_session(user_id)
    uploaded_file = session["uploaded_file"]

    if uploaded_file:
        # 有上傳檔案，使用 RAG 模式
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_uri(
                            file_uri=uploaded_file.uri,
                            mime_type=uploaded_file.mime_type
                        ),
                        types.Part.from_text(text=f"請根據上述提供的檔案內容，用繁體中文回答這個問題：{question}")
                    ]
                )
            ]
        )
        return response.text
    else:
        # 沒有上傳檔案，使用一般多輪對話
        response = session["chat"].send_message(message=question)
        return response.text

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    """處理文字訊息"""
    text = event.message.text
    user_id = event.source.user_id

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        if text.startswith('AI '):
            prompt = text[3:]
            try:
                # 使用 RAG 或一般對話
                reply_text = query_with_rag(user_id, prompt)

                # 檢查是否有上傳檔案，加上提示
                session = get_user_session(user_id)
                if session["uploaded_file"]:
                    reply_text = f"📄 [RAG 模式]\n\n{reply_text}"

                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=reply_text)]
                    )
                )
            except Exception as e:
                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=f"❌ 發生錯誤：{str(e)}")]
                    )
                )
        elif text == "清除文件":
            # 清除使用者上傳的檔案
            session = get_user_session(user_id)
            session["uploaded_file"] = None
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="✅ 已清除上傳的文件，恢復一般對話模式。")]
                )
            )
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="請輸入「AI 問題」來開始對話\n或上傳 TXT/PDF 檔案啟用 RAG 模式")]
                )
            )

@handler.add(MessageEvent, message=FileMessageContent)
def handle_file_message(event):
    """處理使用者上傳的檔案"""
    user_id = event.source.user_id
    file_name = event.message.file_name

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 檢查檔案類型
        if not (file_name.endswith('.txt') or file_name.endswith('.pdf')):
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="⚠️ 目前只支援 TXT 或 PDF 檔案")]
                )
            )
            return

        try:
            # 下載檔案
            file_path = download_line_file(event.message.id, file_name)
            print(f"檔案已下載：{file_path}")

            # 上傳到 Gemini
            uploaded_file = upload_file_to_gemini(file_path)
            print(f"檔案已上傳到 Gemini：{uploaded_file.uri}")

            # 儲存到使用者 session
            session = get_user_session(user_id)
            session["uploaded_file"] = uploaded_file

            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"✅ 檔案「{file_name}」上傳成功！\n\n現在您可以輸入「AI 問題」來詢問關於這份文件的問題。\n\n輸入「清除文件」可恢復一般對話模式。")]
                )
            )
        except Exception as e:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"❌ 檔案處理失敗：{str(e)}")]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Request body: {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[{"type":"message","message":{"type":"file","id":"594237217728102486","markAsReadToken":"ji3Y85pKSqHtrbk-NQcoffPhKfSnk_z4SP0nnGn62KPTqp01ORsWVemwJrTmnZzoFZFtHoDNq4UKH_T3iDSutDjU_o4DZAxvhTBZ4QZUkybNW9OYaSWJGpmtGWhvzTDD34fMDguuc3LIvYGmGemaEhqSU-aZlh1Gl_mGeEFszYyIH3GlSXwVDvLMT1a4tE7kF4Fjj-eQtP28U2a66TyH0w","fileName":"faq.txt","fileSize":21,"contentProvider":{"type":"line"}},"webhookEventId":"01KDNDNQ61W3F822SX88TAC8A9","deliveryContext":{"isRedelivery":false},"timestamp":1767024286829,"source":{"type":"user","userId":"Uaaf8582bee11697441c342939570fcef"},"replyToken":"bee54e27062f41e0b988647da6c79af5","mode":"active"}]}


BODY:  {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[{"type":"message","message":{"type":"file","id":"594237217728102486","markAsReadToken":"ji3Y85pKSqHtrbk-NQcoffPhKfSnk_z4SP0nnGn62KPTqp01ORsWVemwJrTmnZzoFZFtHoDNq4UKH_T3iDSutDjU_o4DZAxvhTBZ4QZUkybNW9OYaSWJGpmtGWhvzTDD34fMDguuc3LIvYGmGemaEhqSU-aZlh1Gl_mGeEFszYyIH3GlSXwVDvLMT1a4tE7kF4Fjj-eQtP28U2a66TyH0w","fileName":"faq.txt","fileSize":21,"contentProvider":{"type":"line"}},"webhookEventId":"01KDNDNQ61W3F822SX88TAC8A9","deliveryContext":{"isRedelivery":false},"timestamp":1767024286829,"source":{"type":"user","userId":"Uaaf8582bee11697441c342939570fcef"},"replyToken":"bee54e27062f41e0b988647da6c79af5","mode":"active"}]}
檔案已下載：/content/uploaded_files/faq.txt
檔案已上傳到 Gemini：https://generativelanguage.googleapis.com/v1beta/files/l5amfbcpcok7


INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 16:04:49] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[{"type":"message","message":{"type":"text","id":"594237255477363098","quoteToken":"kamgK0bCukGTSPzlNrHLvWlUFB9N1je7r8LTBmBYVDOhmNZfsh2Qdej3pWrXE2p_QjnQZjCOZgaA2WRN2P38zLny_ht6VbOU1VfTvKO-KVhxurYGFCh6lW8amSzyw53BZNTy3eqqZv9FOw3cy9mdzw","markAsReadToken":"tXqui6DwsTDNH23RXCs0k-SbJsp47kH49ILa22FaqF5_NddOGX36lxe2d8aCotUq6ikr3SKOma8ctCf6H6gqFPjlrl0GhDVJarZW9idfhv4JXQCaUlLlFR3K6By7qC8yFlyZGxGXvAvqzE1z29ZpFrV_30K6zNmZFNBzuCfuE0h0A-WDC_GprzNK2oTFoRQTBttAxenENiRlW9mIfkQ4pw","text":"AI 請問我是哪一國人"},"webhookEventId":"01KDNDPDHEJNJ7MQ1HACNCHRP9","deliveryContext":{"isRedelivery":false},"timestamp":1767024309299,"source":{"type":"user","userId":"Uaaf8582bee11697441c342939570fcef"},"replyToken":"f238a65faefa46a8ba6c152a29a17c60","mode":"active"}]}


BODY:  {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[{"type":"message","message":{"type":"text","id":"594237255477363098","quoteToken":"kamgK0bCukGTSPzlNrHLvWlUFB9N1je7r8LTBmBYVDOhmNZfsh2Qdej3pWrXE2p_QjnQZjCOZgaA2WRN2P38zLny_ht6VbOU1VfTvKO-KVhxurYGFCh6lW8amSzyw53BZNTy3eqqZv9FOw3cy9mdzw","markAsReadToken":"tXqui6DwsTDNH23RXCs0k-SbJsp47kH49ILa22FaqF5_NddOGX36lxe2d8aCotUq6ikr3SKOma8ctCf6H6gqFPjlrl0GhDVJarZW9idfhv4JXQCaUlLlFR3K6By7qC8yFlyZGxGXvAvqzE1z29ZpFrV_30K6zNmZFNBzuCfuE0h0A-WDC_GprzNK2oTFoRQTBttAxenENiRlW9mIfkQ4pw","text":"AI 請問我是哪一國人"},"webhookEventId":"01KDNDPDHEJNJ7MQ1HACNCHRP9","deliveryContext":{"isRedelivery":false},"timestamp":1767024309299,"source":{"type":"user","userId":"Uaaf8582bee11697441c342939570fcef"},"replyToken":"f238a65faefa46a8ba6c152a29a17c60","mode":"active"}]}


...

INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 16:06:34] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[{"type":"message","message":{"type":"text","id":"594237410331066387","quoteToken":"H7O7gq8ujc4w9lVfklO4mpKez2oN3nNENQeksslXOLkYyrZ1JnkFtikseRg5vI4uYfdOiKhmpQj7MwYhqo7gi1d9iUw9P8imi4LnC3VAb3agPGer39sFRiB_xFK18B68GQv30bm4JBkquxdOsNvY2Q","markAsReadToken":"r4Gw7RZzveI_Lsc73BzKaNf6kDJyGz2QW9KM6UaoY2meML1t7WGVKCvziYeiP5twaDt_JTmyVpusgWncQ6oSp_BxqzMJDqby9tzBpsm2fu-7wMHfDswGdHCed-yC8xLekJ7-nODoIb2xq6bgqd0pH0cXk6bcMs_7EzQlb7i7QdEqF4U-dXfPcA3v_5ic3vbQnmMbDgIQjlNQxVaKNPxWbw","text":"AI 請問我是哪一國人"},"webhookEventId":"01KDNDS7J21WNXQZAXMNJXHNGR","deliveryContext":{"isRedelivery":false},"timestamp":1767024401592,"source":{"type":"user","userId":"Uaaf8582bee11697441c342939570fcef"},"replyToken":"7b47d855741b4b6e9d6641576dedfd42","mode":"active"}]}


BODY:  {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[{"type":"message","message":{"type":"text","id":"594237410331066387","quoteToken":"H7O7gq8ujc4w9lVfklO4mpKez2oN3nNENQeksslXOLkYyrZ1JnkFtikseRg5vI4uYfdOiKhmpQj7MwYhqo7gi1d9iUw9P8imi4LnC3VAb3agPGer39sFRiB_xFK18B68GQv30bm4JBkquxdOsNvY2Q","markAsReadToken":"r4Gw7RZzveI_Lsc73BzKaNf6kDJyGz2QW9KM6UaoY2meML1t7WGVKCvziYeiP5twaDt_JTmyVpusgWncQ6oSp_BxqzMJDqby9tzBpsm2fu-7wMHfDswGdHCed-yC8xLekJ7-nODoIb2xq6bgqd0pH0cXk6bcMs_7EzQlb7i7QdEqF4U-dXfPcA3v_5ic3vbQnmMbDgIQjlNQxVaKNPxWbw","text":"AI 請問我是哪一國人"},"webhookEventId":"01KDNDS7J21WNXQZAXMNJXHNGR","deliveryContext":{"isRedelivery":false},"timestamp":1767024401592,"source":{"type":"user","userId":"Uaaf8582bee11697441c342939570fcef"},"replyToken":"7b47d855741b4b6e9d6641576dedfd42","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 16:06:47] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[{"type":"message","message":{"type":"text","id":"594237458095276296","quoteToken":"_Lu6FTMVHjVQWNU5s7hKgHpbO25CggMCon8Wu0epBrxQlExJW7vXnFHh7xcaYJ9ioAmUKzTd6-T-gCNmqMIVnCwzRDG2fXu_6JYCyaGnZv3rLZ_ouGmmlQ_V-7rMavruAIIPaiYW--1MhedUbioyBw","markAsReadToken":"_ZMtxzxIYArI0EyQ0HibI_Vxqbnflgmy7Y7qrZKuZKyL7nh3m6vAlgsoybu8QsCoq3OBOVtJVmglA-1urOc-2_w9qYbH8VtoxdZKG-FMQeM_PgwCpilQCNlvrZGxn5GMP-VB_8KnGoQMYWe1_2AFDmThgIb6-0tPhWHokep-5jiqoMkYvMTOvMwlZ0dJodRcEVznYIYK9_v0M_Xjc53gCQ","text":"AI 校長是誰"},"webhookEventId":"01KDNDT3EDZPCK4AK1ZF0Q1VHB","deliveryContext":{"isRedelivery":false},"timestamp":1767024430033,"source":{"type":"user","userId":"Uaaf8582bee11697441c342939570fcef"},"replyToken":"80bbc146b07b4692aff1ea245d279b96","mode":"active"}]}


BODY:  {"destination":"U35066f9659160a9cf6dce2b8893e5aae","events":[{"type":"message","message":{"type":"text","id":"594237458095276296","quoteToken":"_Lu6FTMVHjVQWNU5s7hKgHpbO25CggMCon8Wu0epBrxQlExJW7vXnFHh7xcaYJ9ioAmUKzTd6-T-gCNmqMIVnCwzRDG2fXu_6JYCyaGnZv3rLZ_ouGmmlQ_V-7rMavruAIIPaiYW--1MhedUbioyBw","markAsReadToken":"_ZMtxzxIYArI0EyQ0HibI_Vxqbnflgmy7Y7qrZKuZKyL7nh3m6vAlgsoybu8QsCoq3OBOVtJVmglA-1urOc-2_w9qYbH8VtoxdZKG-FMQeM_PgwCpilQCNlvrZGxn5GMP-VB_8KnGoQMYWe1_2AFDmThgIb6-0tPhWHokep-5jiqoMkYvMTOvMwlZ0dJodRcEVznYIYK9_v0M_Xjc53gCQ","text":"AI 校長是誰"},"webhookEventId":"01KDNDT3EDZPCK4AK1ZF0Q1VHB","deliveryContext":{"isRedelivery":false},"timestamp":1767024430033,"source":{"type":"user","userId":"Uaaf8582bee11697441c342939570fcef"},"replyToken":"80bbc146b07b4692aff1ea245d279b96","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [29/Dec/2025 16:07:15] "POST / HTTP/1.1" 200 -
